In [33]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [34]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [35]:
train.head()

Id DetectedCamera  AngleOfSign  \
0  2c9180975a056a64015a1e0a52e57021           Rear          195   
1  2c9180975a056a64015a1e17b32171e4           Rear          203   
2  2c9180975a056a64015a1de4deb16bd5          Front           26   
3  2c9180975a056a64015a1de4deb16bdd           Rear          199   
4  2c9180975a056a64015a1de4deb16bd6           Rear          208   

   SignAspectRatio  SignWidth  SignHeight SignFacing (Target)  
0             1.02         46          45                Rear  
1             1.09         59          54                Rear  
2             0.96        104         108               Front  
3             0.81         38          47                Rear  
4             0.93         54          58                Rear

In [36]:
test.head()

Id DetectedCamera  AngleOfSign  \
0  2c9180975a056a64015a1e10d3f270fe          Right           67   
1  2c9180975a056a64015a1de4deb16bdc          Front           16   
2  2c9180975a056a64015a1e0e70ea70ce          Right           44   
3  2c9180975a056a64015a1dfed0c46ec6          Right           50   
4  2c9180975a056a64015a1dfed0c46ec7          Front           30   

   SignAspectRatio  SignWidth  SignHeight  
0             0.63        107         169  
1             0.88         61          69  
2             1.15        232         202  
3             1.10        137         125  
4             0.95         99         104

In [37]:
train['DetectedCamera'].value_counts()

Front    10910
Right    10516
Left      9298
Rear      7761
Name: DetectedCamera, dtype: int64

In [38]:
# encode as integer
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}
train = train.replace({'DetectedCamera':mapping})
test = test.replace({'DetectedCamera':mapping})

In [39]:
#renaming column
train.rename(columns = {'SignFacing (Target)': 'Target'}, inplace = True)

In [40]:
# encode Target variables based on sample submission file
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}
train = train.replace({'Target': mapping})
                

In [41]:
# target Variable 
y_train = train['Target']
test_id = test['Id']

In [42]:
# drop columns
train.drop(['Target','Id'], inplace= True, axis =1)
test.drop('Id', inplace = True, axis = 1)

In [43]:
print(train.head())
print(test.head())

   DetectedCamera  AngleOfSign  SignAspectRatio  SignWidth  SignHeight
0               3          195             1.02         46          45
1               3          203             1.09         59          54
2               0           26             0.96        104         108
3               3          199             0.81         38          47
4               3          208             0.93         54          58
   DetectedCamera  AngleOfSign  SignAspectRatio  SignWidth  SignHeight
0               1           67             0.63        107         169
1               0           16             0.88         61          69
2               1           44             1.15        232         202
3               1           50             1.10        137         125
4               0           30             0.95         99         104


In [32]:
# train model
clf = RandomForestClassifier(n_estimators = 500, max_features = 3, min_samples_split=5, oob_score =True)
clf.fit(train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [44]:
#predict on test data
pred = clf.predict_proba(test)

In [45]:
# write submission file and submit
columns = ['Front', 'Left', 'Rear','Right']
sub = pd.DataFrame(data = pred, columns = columns)
sub['Id']= test_id
sub = sub[['Id','Front','Left','Rear','Right']]
sub.to_csv("sub_rfih.csv", index=False)